In [45]:
import numpy
from collections import deque
import romania
def adjacency_matrix(graph):
        n = len(graph.nodes)

        nodes = list(graph.nodes.keys())
        adj_matrix = [[0 for i in range(n)] for j in range(n)]
        
        for node in graph.nodes.values():
            row_index = nodes.index(node.name)
            for neighbor,weight in node.neighbours:
                col_index = nodes.index(neighbor)
                adj_matrix[row_index][col_index] = 1
        
        
        return adj_matrix,nodes


[[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 

([[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

A function that converts the graph expression to adjecency Matrix

In [113]:
def degree_centrality(graph):
        degree_centralities = {}
        number_of_nodes = len(graph.nodes)
        
       
        for node in graph.nodes:
            degree_centralities[node] = len(graph.nodes[node].neighbours)/number_of_nodes
        
        top_degree_centralities = []
        top = max(degree_centralities.values())

        for centrality in degree_centralities:
            if degree_centralities[centrality] == top:
                top_degree_centralities.append(centrality)
        

        return degree_centralities,top_degree_centralities
print(degree_centrality(romania.romania))

({'Oradea': 0.1, 'Zerind': 0.1, 'Arad': 0.15, 'Timisoara': 0.1, 'Lugoj': 0.1, 'Mehadia': 0.1, 'Drobeta': 0.1, 'Craiova': 0.15, 'Sibiu': 0.2, 'Rimnicu_Vilcea': 0.15, 'Fagaras': 0.1, 'Pitesti': 0.15, 'Giurgiu': 0.05, 'Bucharest': 0.2, 'Urziceni': 0.15, 'Eforie': 0.05, 'Hirsova': 0.1, 'Vaslui': 0.1, 'Iasi': 0.1, 'Neamt': 0.05}, ['Sibiu', 'Bucharest'])


A function that calculates degree centrality

In [132]:


def closeness_centrality(graph):
    node_names=[node for node in graph.nodes]
    closeness_scores = [0 for i in range(len(node_names))]
    
    close=0
    for i in range(len(node_names)):
        distances = {node: float('inf') for node in graph.nodes}
        distances[node_names[i]] = 0 
        
        queue = [node_names[i]]
        while queue:
            curr_node = queue.pop(0)
            for neighbor,cost in graph.nodes[curr_node].neighbours:
                total_dist= distances[curr_node] + cost
                if distances[neighbor] == float('inf'):
                    distances[neighbor]=total_dist
                    queue.append(neighbor)
                if distances[neighbor]>total_dist:
                    distances[neighbor]=total_dist
        
        distance = distances
        sum_distances = sum(distance.values())
        closeness_scores[i] = (len(node_names) - 1) / sum_distances
        if closeness_scores[i]>close:
            close=closeness_scores[i]
    top_ranked = []
    maximum = max(closeness_scores)
    for i in range(len(closeness_scores)):
        if closeness_scores[i] == maximum:
            top_ranked.append((node_names[i], closeness_scores[i]))
            
    return closeness_scores, top_ranked
print(closeness_centrality(romania.romania))

([0.002332433095998036, 0.002197547999074717, 0.00255170561375235, 0.002170436371944254, 0.0021687022029448695, 0.002287778446718844, 0.0024733142410830514, 0.002938447262604392, 0.003023070803500398, 0.003294607248135946, 0.002819409407924024, 0.0034289839379173436, 0.0025242460475621096, 0.003216522769595395, 0.002884469409442842, 0.0019581572709471296, 0.0023298589822194973, 0.002215743440233236, 0.0018911117746591023, 0.0016360974769654697], [('Pitesti', 0.0034289839379173436)])


A function that calculates closeness centrality.

In [131]:


def eigenvector_centrality(graph, max_iterations=100, tolerance=1e-6):
   
    adj_matrix, node_names = adjacency_matrix(graph)
    
    centrality_values = numpy.ones(len(adj_matrix))
    centrality_values /= numpy.linalg.norm(centrality_values)

    for _ in range(max_iterations):
        new_centrality = numpy.dot(adj_matrix, centrality_values)

        new_centrality /= numpy.linalg.norm(new_centrality)

        if numpy.linalg.norm(new_centrality - centrality_values, 2) < tolerance:
            break

        centrality_values = new_centrality

    max_centrality = max(centrality_values)
    top_centrality_nodes = []
    for i in range(len(centrality_values)):
        if centrality_values[i] == max_centrality:
            top_centrality_nodes.append(node_names[i])

    return centrality_values, top_centrality_nodes

print(eigenvector_centrality(romania.romania))

[[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 

A function that calculates eiganvector centrality.

In [125]:
import numpy

def pagerank_centrality(graph, damping_factor=0.85, max_iterations=100, tolerance=1e-6):
    adj_matrix, node_names = adjacency_matrix(graph)
    adj_matrix = numpy.array(adj_matrix)
    num_nodes = len(adj_matrix)
    pagerank_scores = numpy.ones(num_nodes) / num_nodes

    for _ in range(max_iterations):
        row_sums = adj_matrix.sum(axis=1, keepdims=True)
        transition_matrix = numpy.where(numpy.logical_and(row_sums != 0, ~numpy.isnan(row_sums), ~numpy.isnan(adj_matrix)), adj_matrix / row_sums, 1 / num_nodes)
        new_pagerank_scores = (1 - damping_factor) / num_nodes + damping_factor * numpy.dot(transition_matrix.T, pagerank_scores)
        if numpy.linalg.norm(new_pagerank_scores - pagerank_scores, 2) < tolerance:
            break
        pagerank_scores = new_pagerank_scores
  
    max_centrality = max(pagerank_scores)
    top_centrality_nodes = []
    for i in range(len(pagerank_scores)):
        if pagerank_scores[i] == max_centrality:
            top_centrality_nodes.append(node_names[i])

    return pagerank_scores, top_centrality_nodes
print(pagerank_centrality(romania.romania))


[[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 

A function that calculates page rank centrality.

In [129]:


def katz_centrality(graph, alpha=0.1, beta=1.0, max_iter=100, tol=1e-6):
    adj_matrix, nodes = adjacency_matrix(graph)
    n = len(adj_matrix)
    
  
    ketz_centrality = numpy.zeros(n)
    beta = numpy.full(n, beta)

    for _ in range(max_iter):
        
        new_ketz_centrality = alpha * numpy.dot(adj_matrix, ketz_centrality) + beta

        if numpy.linalg.norm(new_ketz_centrality - ketz_centrality, 2) < tol:
            break

        ketz_centrality = new_ketz_centrality

    ketz_centrality /= numpy.linalg.norm(ketz_centrality)

    top_centrality_nodes = []
    maximum = max(ketz_centrality)
    for i in range(len(ketz_centrality)):
        if ketz_centrality[i] == maximum:
            top_centrality_nodes.append((nodes[i], ketz_centrality[i]))
            
    return ketz_centrality, top_centrality_nodes
print(katz_centrality(romania.romania))


[[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 

A function that calculates katz centrality.

In [130]:


def shortest_paths(graph):
    nodes = list(graph.nodes.keys())
    n = len(nodes)
    dist = numpy.full((n, n), numpy.inf)

 
    for i, node in enumerate(nodes):
        for neighbor, weight in graph.nodes[node].neighbours:
            j = nodes.index(neighbor)
            dist[i, j] = weight

    for k in range(n):
        for i in range(n):
            for j in range(n):
                dist[i, j] = min(dist[i, j], dist[i, k] + dist[k, j])

    return dist


def betweenness_centrality(graph):
    nodes = list(graph.nodes.keys())
    n = len(nodes)
    dist = shortest_paths(graph)
    betweenness = numpy.zeros(n)

    for s in range(n):
        stack = []
        predecessors = [[] for _ in range(n)]
        num_shortest_paths = numpy.zeros(n)
        distance = numpy.full(n, -1)
        distance[s] = 0
        num_shortest_paths[s] = 1

        queue = [s]
        while queue:
            v = queue.pop(0)
            stack.append(v)
            for neighbor, _ in graph.nodes[nodes[v]].neighbours:
                w = nodes.index(neighbor)
                if distance[w] < 0:
                    queue.append(w)
                    distance[w] = distance[v] + 1
                if distance[w] == distance[v] + 1:
                    num_shortest_paths[w] += num_shortest_paths[v]
                    predecessors[w].append(v)

        delta = numpy.zeros(n)
        while stack:
            w = stack.pop()
            for v in predecessors[w]:
                delta[v] += (num_shortest_paths[v] / num_shortest_paths[w]) * (1 + delta[w])
            if w != s:
                betweenness[w] += delta[w]

    max_betweenness = numpy.max(betweenness)
    if max_betweenness > 0:
        betweenness /= max_betweenness

    top_betweenness_centrality_nodes = [nodes[i] for i in range(n) if betweenness[i] == numpy.max(betweenness)]

    return betweenness, top_betweenness_centrality_nodes
print(betweenness_centrality(romania.romania))



(array([0.07400722, 0.0198556 , 0.35379061, 0.14981949, 0.07400722,
       0.10649819, 0.21119134, 0.33754513, 0.64981949, 0.16064982,
       0.4765343 , 0.40433213, 0.        , 1.        , 0.82310469,
       0.        , 0.19494585, 0.36823105, 0.19494585, 0.        ]), ['Bucharest'])


A function that calculates betweeness centrality.